# Split Decisions

Solver / Game creator aid for the NYT 'Split Decisions' puzzle. FInds word pairs that can be created using shared word shapes with different letter pairs, as in the example pair below (SCREW / SINEW)

- '_SC_ _'
- '_IN_ _'


In [102]:
import numpy as np
import pandas as pd
import timeit
import matplotlib.pyplot as plt
%matplotlib inline

## Solver, given initial positions

In [60]:
#corncob = r"D:\Users\hiltss\Github\SplitDecisions_Wordgame\corncob_lowercase.txt"
#wiki = r"D:\Users\hiltss\Github\SplitDecisions_Wordgame\wiki-100k.txt"
#big = r"D:\Users\hiltss\Github\SplitDecisions_Wordgame\words.txt"

corncob = r"C:\Users\stefa\Documents\GitHub\SplitDecisions_Wordgame-master\corncob_lowercase.txt"
#wiki = r"C:\Users\stefa\Documents\GitHub\SplitDecisions_Wordgame-master\wiki-100k.txt"
big = r"C:\Users\stefa\Documents\GitHub\SplitDecisions_Wordgame-master\words.txt"

word_list_corncob = [x.upper() for x in open(corncob).read().split("\n")]
#word_list_wiki = open(wiki, encoding='utf-8').read().split("\n")
word_list = open(big).read().split("\n")

## Create dictionaries of word pairs
Counts of available words by:
- Word length
- Letter pair
- Position

Create cross-pair counts by position, length (can actually be length-agnostic for some purposes; length is optional). 


### Create list of letter pairs, calc frequency
- Defines alphabet
- Creates letter pairs from alphabet
- Reduces pairs list based on frequency in dictionary. 

In [84]:
alphabet = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [78]:
start = timeit.default_timer()
df_freq = pd.DataFrame()

for pair in pairs_list:
    freq=0
    for word in word_list_corncob:
        if word.find(pair)>-1:
            freq = freq+1
    df_temp = pd.DataFrame(np.column_stack([pair,freq]))
    df_freq = df_freq.append(df_temp)
    
    
stop = timeit.default_timer()
print(stop-start)

11.70361751943284


In [123]:
df_freq.columns = ['Pair','Freq']
df_freq['Freq'] = pd.to_numeric(df_freq['Freq']) 
pairs_list_scrub = df_freq[df_freq['Freq']>1000]['Pair'].values.tolist()
len(pairs_list_scrub)


125

###  Define Calc functions

In [35]:
def make_word_match(shape, word_list):
    blank_pos = []
    list_shape=[]
    for x in range(len(shape)):
        blank_pos.append(shape.find('_',x))
    blank_pos = list(set(blank_pos))
    letter_pos = [x for x in range(len(shape)) if x not in blank_pos]
    word_list_len = [x.upper() for x in word_list if len(x)==len(shape)]
    for word in word_list_len:
        check = 0 
        for x in range(len(shape)):
            if x in letter_pos:
                if word[x]!=shape[x]:
                    check = check+1
        if check == 0:
            #list_a.append(word)
            list_shape.append(''.join([word[x] for x in range(len(word)) if x in blank_pos]))
    return list_shape, blank_pos

def make_shape(pos, length, pair):
    shape = []
    for x in range(length):
        if x==pos:
            shape.append(pair[0])
        elif x==pos+1:
            shape.append(pair[1])
        else:
            shape.append('_')
    return ''.join(shape)

def match_list(list_a,list_b,shape_a,shape_b,blank_pos):
    match = list(set(list_b)&set(list_a))
    comb_list = []
    for word in match:
        word_pair_a = []
        word_pair_b = []
        count = 0
        for x in range(len(shape_a)):
            if x in blank_pos:
                word_pair_a.append(word[count])
                word_pair_b.append(word[count])
                count=count+1
            else: 
                word_pair_a.append(shape_a[x])
                word_pair_b.append(shape_b[x])
        
        comb_list.append([''.join(word_pair_a),''.join(word_pair_b)])
    return comb_list
        
def run_whole(pair_a,pair_b,pos,length,word_list):
    shape_a = make_shape(pos,length,pair_a)
    shape_b = make_shape(pos,length,pair_b)

    list_a,blank_pos = make_word_match(shape_a,word_list)
    list_b,blank_pos = make_word_match(shape_b,word_list)

    return match_list(list_a,list_b,shape_a,shape_b,blank_pos)



### Run stuff
- Loop through pairs in scrubbed pairs list to create dataframe of counts of words viable word pairs. 
- Function creates DF for word length and pair position

In [126]:
start = timeit.default_timer()
df_pairs = pd.DataFrame()

for pair_a in pairs_list_scrub[:10]:
    for pair_b in pairs_list_scrub[:10]:
        if pair_b == pair_a:
            pass
        else:
            pairmatch = len(run_whole(pair_a,pair_b,pos=1,length=6,word_list=word_list_corncob))
            df_temp = pd.DataFrame(np.column_stack([pair_a,pair_b,pairmatch]))
            df_pairs = df_pairs.append(df_temp)
            
stop = timeit.default_timer()
print(stop-start)

3.691044386069734


In [128]:
125*125/100*3.7

578.125